In [20]:
from hashlib import new
import numpy as np 
import random
import math
#GLOBAL VARS
#size
RowCount  = 6
ColCount = 7
#turn
Player = 0
AI = 1

#FUNCTIONS

def create_board():
  board = np.zeros ((RowCount,ColCount))
  return board

def drop_piece(board, r, col, piece):
  board[r][col] = piece
  return r, col

def valid_loc(board, col):
  return board[RowCount - 1][col] == 0

def getnext(board, col):
  for r in range (RowCount):
    if board [r][col] ==0:
      return r

def boardgfx(board):
  print(np.flip(board, 0))

def wincondition(board, piece):
  #horizontal
  for c in range(ColCount - 3):
    for r in range(RowCount):
      if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
        return True
  #vertical   
  for c in range(ColCount):
   for r in range(RowCount - 3):
     if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
       return True

	# upore diagonal
  for c in range(ColCount-3):
    for r in range(RowCount-3):
      if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
        return True

	# niche diagonal
  for c in range(ColCount-3):
    for r in range(3, RowCount):
      if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
        return True

def score(board, piece):
  score = 0

  #center
  centerarray = [int(i) for i in list(board[:, ColCount//2])]
  centercount = centerarray.count(piece)
  score += centercount * 3
  #Horizontal
  for r in range(RowCount):
    row_array = [int(i)for i in list(board[r,:])]
    for c in range(ColCount-3):
      window = row_array[c:c+4]     
      score += windoweval(window, piece)
  
  #vertical
  for c in range(ColCount):
    col_array = [int(i) for i in list (board[:, c])]
    for r in range(RowCount-3):
      window = col_array[r:r+4]
      score += windoweval(window, piece)

  #upore diagonal
  for r in range(RowCount-3):
    for c in range(ColCount-3):
      window = [board[r+i][c+i] for i in range(4)]      
      score += windoweval(window, piece)
  
  #niche diagonal
  for r in range(RowCount-3):
    for c in range(ColCount-3):
      window = [board[r+3-i][c+i] for i in range(4)] 
      score += windoweval(window, piece)

  return score

def windoweval(window, piece):
  score = 0
  opiece = 1
  if piece == 1:
    opeice = 2

  if window.count(piece) == 4:
    score += 100
  elif window.count(piece) == 3 and window.count(0)==1:
    score += 10
  elif window.count(piece) == 2 and window.count(0) == 2:
    score +=5
  if window.count(opiece) == 3 and window.count(0) == 1:
    score -= 7
  return score

def endpoint(board):
  return wincondition(board, 1) or wincondition(board, 2) or len(getValidLoc(board)) == 0

def minimax(board, d, alpha, beta, maxplayer):
  validloc = getValidLoc(board)
  endpointflag = endpoint(board)
  if d == 0 or endpointflag:
    if endpointflag:
      #ai win
      if wincondition(board, 2):
        return (None, 999*999)
      #player win
      elif wincondition(board, 1):
        return (None, -999*999)
      #end game
      else: 
        return (None,0)
    else:
      return (None, score(board, 2))
  #min condition
  if maxplayer:
    value = -math.inf
    column = random.choice(validloc)
    for col in validloc:
      row = getnext(board, col)
      board2 = board.copy()
      drop_piece(board2, row, col , 2)
      newscore = minimax(board2, d-1, alpha, beta, False)[1]
      if newscore>value:
        value = newscore
        column = col
      alpha = max(value, alpha)
      if alpha >= beta:
        break
    return  column, value
  #max condition
  else:
    value = math.inf
    column = random.choice(validloc)
    for col in validloc:
      row = getnext(board, col)
      board2 = board.copy()
      drop_piece(board2, row, col, 1)
      newscore = minimax(board2, d-1, alpha, beta, True)[1]
      if newscore<value:
        value = newscore
        column = col
      beta = min(beta, value)
      if alpha>= beta:
        break
    return  column, value
    
def getValidLoc(board):
  validlocations = []
  for col in range(ColCount):
    if valid_loc(board, col):
      validlocations.append(col)
  return validlocations

def AImove(board, piece):
  validlocations = getValidLoc(board)
  bestscore = -12345
  bestcol = random.choice(validlocations)
  for col in validlocations:
    row = getnext(board, col)
    board2 = board.copy()
    drop_piece(board2, row, col, piece)
    thescore = score(board2, piece)
    if thescore>bestscore:
      bestscore = thescore
      bestcol = col
  
  return bestcol


#CODE BEGIN

board = create_board()
boardgfx(board)
gover = False
turn = random.randint(Player, AI)

while not gover:
  #player1 input
  if turn ==Player:
    col = int(input("Player 1 select (0-6)"))

    if valid_loc (board, col):
      row = getnext(board, col)
      print(drop_piece(board, row, col , 1))
    else:
      print("INVALID INPUT")
      break
    
    if wincondition(board, 1):
      boardgfx(board)
      print ("PLAYER 1 wins!")
      break
  #AI input
  else:
    #col = AImove(board, 2)
    #chose depth as 4 because 6 would take too long time while 5 is simply better 
    col, aiscore = minimax(board, 5, -math.inf, math.inf, True)
    if valid_loc (board, col):
      row = getnext(board, col)
      print(drop_piece(board, row, col , 2))
 
    if wincondition(board, 2):
      boardgfx(board)
      print ("PLAYER 2 wins!")
      break
  
  boardgfx(board)

  turn = turn + 1
  turn = turn % 2
  


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
Player 1 select (0-6)2
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
Player 1 select (0-6)2
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
Player 1 select (0-6)3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0.